In [1]:
import numpy as np
import pandas as pd

In [2]:
TRAIN = 'train.csv'
TEST = 'test.csv'
df_correct_file = 'df_correct.csv'

In [3]:
df = pd.read_csv(TRAIN)
df_correct = pd.read_csv(df_correct_file)

In [12]:
X = pd.read_csv('x_train.csv')
X_train=X.to_numpy()
X_test = pd.read_csv('x_test.csv')
y = pd.read_csv('y_train.csv')
y_train = y.to_numpy()

In [9]:
class RegressionTreeFastMse():

    
    # объявляем характеристики класса
    def __init__(self, max_depth=8, min_size=5):
        
        self.max_depth = max_depth
        self.min_size = min_size
        self.value = 0
        self.feature_idx = -1
        self.feature_threshold = 0
        self.left = None
        self.right = None
        
    # процедура обучения - сюда передается обучающая выборка
    def fit(self, X, y):
        
        # начальное значение - среднее значение y
        self.value = y.mean()
        # начальная ошибка - mse между значением в листе (пока нет
        # разбиения, это среднее по всем объектам) и объектами
        base_error = ((y - self.value) ** 2).sum()
        error = base_error
        flag = 0
        
        # пришли в максимальную глубину
        if self.max_depth <= 1:
            return
    
        dim_shape = X.shape[1]
        
        left_value, right_value = 0, 0
        
        for feat in range(dim_shape):
            
            prev_error1, prev_error2 = base_error, 0 
            idxs = np.argsort(X[:, feat])
            
            # переменные для быстрого переброса суммы
            mean1, mean2 = y.mean(), 0
            sm1, sm2 = y.sum(), 0
            
            N = X.shape[0]
            N1, N2 = N, 0
            thres = 1
            
            while thres < N - 1:
                N1 -= 1
                N2 += 1

                idx = idxs[thres]
                x = X[idx, feat]
                
                # вычисляем дельты - по ним в основном будет делаться переброс
                delta1 = (sm1 - y[idx]) * 1.0 / N1 - mean1
                delta2 = (sm2 + y[idx]) * 1.0 / N2 - mean2
                
                # увеличиваем суммы
                sm1 -= y[idx]
                sm2 += y[idx]
                
                # пересчитываем ошибки за O(1)
                prev_error1 += (delta1**2) * N1 
                prev_error1 -= (y[idx] - mean1)**2 
                prev_error1 -= 2 * delta1 * (sm1 - mean1 * N1)
                mean1 = sm1/N1
                
                prev_error2 += (delta2**2) * N2 
                prev_error2 += (y[idx] - mean2)**2 
                prev_error2 -= 2 * delta2 * (sm2 - mean2 * N2)
                mean2 = sm2/N2
                
                # пропускаем близкие друг к другу значения
                if thres < N - 1 and np.abs(x - X[idxs[thres + 1], feat]) < 1e-5:
                    thres += 1
                    continue
                
                # 2 условия, чтобы осуществить сплит - уменьшение ошибки 
                # и минимальное кол-о эл-в в каждом листе
                if (prev_error1 + prev_error2 < error):
                    if (min(N1,N2) > self.min_size):
                    
                        # переопределяем самый лучший признак и границу по нему
                        self.feature_idx, self.feature_threshold = feat, x
                        # переопределяем значения в листах
                        left_value, right_value = mean1, mean2

                        # флаг - значит сделали хороший сплит
                        flag = 1
                        error = prev_error1 + prev_error2
                                     
                thres += 1
 
        # ничего не разделили, выходим
        if self.feature_idx == -1:
            return
        
        self.left = RegressionTreeFastMse(self.max_depth - 1)
        # print ("Левое поддерево с глубиной %d"%(self.max_depth - 1))
        self.left.value = left_value
        self.right = RegressionTreeFastMse(self.max_depth - 1)
        # print ("Правое поддерево с глубиной %d"%(self.max_depth - 1))
        self.right.value = right_value
        
        idxs_l = (X[:, self.feature_idx] > self.feature_threshold)
        idxs_r = (X[:, self.feature_idx] <= self.feature_threshold)
    
        self.left.fit(X[idxs_l, :], y[idxs_l])
        self.right.fit(X[idxs_r, :], y[idxs_r])
        
    def __predict(self, x):
        if self.feature_idx == -1:
            return self.value
        
        if x[self.feature_idx] > self.feature_threshold:
            return self.left.__predict(x)
        else:
            return self.right.__predict(x)
        
    def predict(self, X):
        y = np.zeros(X.shape[0])
        
        for i in range(X.shape[0]):
            y[i] = self.__predict(X[i])    
        return y
    
    
    


class g_boost():   
    def __init__(self,n_trees=10, max_depth=3, eta=1,min_size=5):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.eta = eta
        self.min_size = min_size
        self.trees = []      
   
    def bias(self,y, z):
        return (y - z)
    
    def mean_squared_error(self,y_real, prediction):
        return (sum((y_real - prediction)**2)) / len(y_real)
    
    def R2(self,y,y_pred):
        return 1- ((y_pred - y)**2).sum()/((y-y.mean())**2).sum()
      
    def predict(self,X):
        # Реализуемый алгоритм градиентного бустинга будет инициализироваться нулевыми значениями,
        # поэтому все деревья из списка trees_list уже являются дополнительными и при предсказании прибавляются с шагом eta
        y_pred = np.zeros(X.shape[0])
        for alg in self.trees:
            y_pred+=self.eta * alg.predict(X)     
        return y_pred
     
    def fit(self,X_train,y_train): 
        self.X_train = X_train
        self.y_train = y_train
    # Деревья будем записывать в список
        for i in range(self.n_trees):
            tree = RegressionTreeFastMse(max_depth=self.max_depth,min_size = self.min_size)
            # инициализируем бустинг начальным алгоритмом, возвращающим ноль, 
            # поэтому первый алгоритм просто обучаем на выборке и добавляем в список
            if len(self.trees) == 0:
                # обучаем первое дерево на обучающей выборке
                tree.fit(X_train, y_train)
            else:
                # Получим ответы на текущей композиции
                target = self.predict(X_train)          
                # алгоритмы начиная со второго обучаем на сдвиг
                tree.fit(X_train, self.bias(y_train, target))        
            self.trees.append(tree)
    
    def  train_test(self,metrika = 'r2'):
        if metrika!='r2':
            return self.mean_squared_error(self.y_train,self.predict(self.X_train))
        else:
            return self.R2(self.y_train,self.predict(self.X_train))
    
    def metrika_test(self, X_test,y_test, metrika = 'r2'):
        if metrika!='r2':
            return self.mean_squared_error(y_test,self.predict(X_test))
        else:
            return self.R2(y_test,self.predict(X_test))
        
def cross_validation(n,size):
        data=[]
        r = np.arange(n)
        for i in range(size-1):
            ind = np.random.choice(r,size=int(n/size), replace=False)
            data.append(ind)
            r=np.setdiff1d(r,ind) 
        data.append(r)
        return data
    
def test_cros(X,y,size=5,print_res =True,n_trees=10,max_depth=8,eta=0.5,min_size=5):
        cros_val = cross_validation(X.shape[0],size)
        data=[]
        for i in range(len(cros_val)):
            ind_train = np.hstack((cros_val[:i]+cros_val[i+1:]))
            ind_test = cros_val[i]
            forest = g_boost(n_trees=n_trees, max_depth=max_depth, eta=eta,min_size=min_size)
            forest.fit(X[ind_train],y[ind_train])
            r2_test = forest.metrika_test(X[ind_test],y[ind_test])             
            r2_train = forest.train_test()
            data.append((r2_test,r2_train))
            if print_res:
                print(f'Выборка номер {i} r2 на обучении {r2_train}, r2 на тесте {r2_test}')              
        return data             

В результате огромного тмножества эксперементов над параметрами, гиперпаораметрами и
набором данных были подобраны лучшие
n_trees=50,max_depth=5,eta=0.2,min_size=5


In [13]:
test =test_cros(X_train,np.ravel(y_train),n_trees=50,max_depth=5,eta=0.2, min_size=5)

Выборка номер 0 r2 на обучении 0.7934621941146989, r2 на тесте 0.7859003533647789
Выборка номер 1 r2 на обучении 0.7917654745451445, r2 на тесте 0.7838205327776654
Выборка номер 2 r2 на обучении 0.794826725983474, r2 на тесте 0.7755099495962103
Выборка номер 3 r2 на обучении 0.7941052845207411, r2 на тесте 0.7758350297608734
Выборка номер 4 r2 на обучении 0.7958767574971524, r2 на тесте 0.776769654211275
